In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1603981895563_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [40]:
from pyspark.sql.types import *

# Define schema for movies dataset
moviesStruct = [StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)]

moviesSchema = StructType(moviesStruct)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Read movies from HDFS
moviesDf = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(moviesSchema) \
    .load("hdfs:///user/hadoop/movielens/movies.csv")

moviesDf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows

In [5]:
# Define schema for ratings dataset
ratingsStruct = [StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", IntegerType(), True)]

ratingsSchema = StructType(ratingsStruct)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Read ratings from HDFS
ratingsDf = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(ratingsSchema) \
    .load("hdfs:///user/hadoop/movielens/ratings.csv")

ratingsDf.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27753444

In [52]:
sampleDf = moviesDf.sample(fraction=0.001)

# sampleDf.show(100, True)
for row in sampleDf.rdd.collect():
    print("Please rate the following movie (1-5 (best), or 0 if not seen):\n" + row.title + ":")
    rate = input()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
EOF when reading a line
Traceback (most recent call last):
EOFError: EOF when reading a line



In [42]:
joinDf = moviesDf.join(ratingsDf, "movieId")
joinDf.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'ratingsDf' is not defined
Traceback (most recent call last):
NameError: name 'ratingsDf' is not defined



In [9]:
# To summarize :
usersDf = ratingsDf.select("userId").distinct()

# Got 1000209 ratings from 6040 users on 3883 movies.
print("Got {} ratings from {} users on {} movies.".format(ratingsDf.count(), usersDf.count(), moviesDf.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Got 27753444 ratings from 283228 users on 58098 movies.

In [10]:
# Splitting training data
training,test = ratingsDf.randomSplit([0.8, 0.2], seed=12345)

# Got 1000209 ratings from 6040 users on 3883 movies.
print("Training {}, test {}.".format(training.count(), test.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training 22202834, test 5550610.

In [11]:
# Training the data
from pyspark.ml.recommendation import ALS

"""
As the Netflix Prize competition has demonstrated, matrix factorization models are superior 
to classic nearest-neighbor techniques for producing product recommendations, 
allowing the incorporation of additional information such as implicit feedback, temporal effects, and confidence levels.
"""
als = ALS(maxIter=5,
          regParam=0.01, 
          implicitPrefs=False, 
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="rating", 
          coldStartStrategy="drop")

model = als.fit(training)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Evaluate the model by computing the RMSE on the test data
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error = 0.8430091116019851

In [13]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Generate top 10 movie recommendations for a specified set of users
users = ratingsDf.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
movies = ratingsDf.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql.functions import col

# Recommendations for a specific user
aUserId = 12
recommandations = userRecs.filter(col("userId") == aUserId)
recommandations.show(1, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                             |
+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|12    |[[188925, 18.428097], [188923, 16.585285], [182521, 11.055481], [160824, 10.536652], [188935, 9.214048], [141532, 9.110452], [163713, 8.989071], [178027, 8.866638], [121650, 8.762271], [153184, 8.459264]]|
+------+----------------------------------------------------------------------------------------------------------------------------------------

In [16]:
from pyspark.sql.functions import explode

# Let's flatten the movie recommandations and look in detail
userRecommandations = recommandations.select(
  explode(col("recommendations").movieId).alias("movieId")
)

print("Recommandations for user {} :".format(aUserId))

moviesDf.join(userRecommandations, "movieId").show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Recommandations for user 12 :
+-------+----------------------------------+---------------------+
|movieId|title                             |genres               |
+-------+----------------------------------+---------------------+
|188925 |8 Murders a Day (2011)            |(no genres listed)   |
|188923 |49 Pulses (2017)                  |(no genres listed)   |
|182521 |Thakara (1979)                    |(no genres listed)   |
|160824 |W.A.R.: Women Against Rape (1987) |Drama|Thriller       |
|188935 |Earth's Natural Wonders (2016)    |Documentary          |
|141532 |Retrieval (2006)                  |(no genres listed)   |
|163713 |Rampage: Capital Punishment (2014)|Action|Crime|Thriller|
|178027 |People That Are Not Me (2016)     |Drama|Romance        |
|121650 |The Loyal 47 Ronin (1958)         |Drama                |
|153184 |Vergeef me                        |(no genres listed)   |
+-------+----------------------------------+---------------------+

In [18]:
# Ratings from the user
ratingsDf.filter(col("userId") == aUserId) \
    .join(moviesDf, "movieId") \
    .sort(col("rating").desc()) \
    .show(100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+----------+-------------------------------------+----------------------------------------+
|movieId|userId|rating|timestamp |title                                |genres                                  |
+-------+------+------+----------+-------------------------------------+----------------------------------------+
|1090   |12    |3.5   |1108087090|Platoon (1986)                       |Drama|War                               |
|2502   |12    |3.5   |1108087157|Office Space (1999)                  |Comedy|Crime                            |
|1380   |12    |3.0   |1108087105|Grease (1978)                        |Comedy|Musical|Romance                  |
|45     |12    |3.0   |1108087163|To Die For (1995)                    |Comedy|Drama|Thriller                   |
|2001   |12    |3.0   |1108087154|Lethal Weapon 2 (1989)               |Action|Comedy|Crime|Drama               |
|2406   |12    |3.0   |1108087093|Romancing the Stone (1984)           |Action|Adventure